앨런 튜링의 튜링머신과 마찬가지로 람다계산법은 
수학자 다비드 힐베르트가 제시한 수리명제자동판별문제(Entscheidungsproblem)의
해결 불가능함을 보이기 위해 알론조 처치가 최소한의 문법으로 고안한
이론적 소품으로써 탄생했다. 이후 람다계산법은 프로그래밍 언어 이론의 연구에
활발히 활용되고 있으며 특히 함수형 프로그래밍언어 설계의 기초가 되고 있다.

이론적으로는 순수한 람다계산법만으로도 조건문, 자연수 등을 포함한 모든 계산이 표현 가능지만
실제 컴퓨터에서 그런 방식으로 계산한다면 비효율적일 것이다.
람다계산법을 기초로 하여 컴퓨터 하드웨어에서 제공하는 정수 연산 등
여러 가지 다양한 기능을 덧붙여 나간 언어들이 바로 함수형 언어다.
또한, 최근에는 함수형 언어로 시작하지 않은 많은 프로그래밍언어들도
람다계산법의 문법이나 개념을 도입하고 있기도 하므로 실무적 프로그래밍
능력 향상을 위해서도 람다계산법을 바탕으로 하는 언어에 대한 기본적 이해가 필요하다.

이 장에서는 지금까지 살펴본 덧셈식과 람다식 그리고 조건식까지 포함하는
언어에 대한 인자 먼저 값계산(call-by-value evaluation)을 하는
인터프리터를 하스켈로 작성해 본다.

\newpage

\begin{align*}
x &\;\in\; \texttt{Nm} \qquad\qquad n \;\in\; \mathbb{Z} \\
e &\;\in \texttt{Expr}
    ~::=~ x ~\mid~ \lambda x.e ~\mid~ e_1~e_2 
    ~\mid~ n ~\mid~ e_1 +\, e_2
    ~\mid~ \textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0
\\[-3em]
\end{align*}
\begin{align*}
\texttt{Env} &~=~ \texttt{Nm} \xrightharpoonup{_\textrm{fin}} \texttt{Val} & \quad
\rho &\in \texttt{Env} ~ ::= ~ \{x_1{\mapsto}v_1,\,\ldots,\,x_n{\mapsto}v_n\} \\
\texttt{Val} &~\subset~ (\texttt{Expr}\times\texttt{Env}) \;\cup\; \mathbb{Z} & \quad
v &\in \texttt{Val} ~ ::= ~ \langle \lambda x.e, \rho \rangle ~\mid~ n \\
\cdot\Longmapsto\cdot &~\subset~ (\texttt{Expr}\times\texttt{Env})\;\times\;\texttt{Val}
\\[-3em]
\end{align*}

$$
\langle x,\rho\rangle \Longmapsto \rho(x)
\qquad
\langle\lambda x.e,\rho\rangle \Longmapsto \langle\lambda x.e,\rho\rangle
$$
$$
\frac{
      \langle e_1,\rho\rangle \Longmapsto \langle\lambda x.e,\rho_1\rangle
      \quad
      \langle e_2,\rho\rangle \Longmapsto v_2
      \quad
      \langle e,\{x{\mapsto}v_2\}\rho_1\rangle \Longmapsto v
    }{\langle e_1~e_2,\rho\rangle \Longmapsto v}
$$
$$
\langle n,\rho\rangle \Longmapsto n
\qquad
\frac{\langle e_1,\rho\rangle \Longmapsto n_1 
      \quad
      \langle e_2,\rho\rangle \Longmapsto n_2}{
        \langle e_1 +\, e_2,\rho\rangle \Longmapsto n}(n\equiv n_1+n_2)
$$
$$
\frac{\langle e,\rho\rangle \Longmapsto 0
      \quad
      \langle e_0,\rho\rangle \Longmapsto n_0}{
        \langle\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0,\,\rho\rangle \Longmapsto n_0}
\quad        
\frac{\langle e,\rho\rangle \Longmapsto n
      \quad
      \langle e_1,\rho\rangle \Longmapsto n_1}{
        \langle\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0,\,\rho\rangle \Longmapsto n_1}(n\neq 0)
$$

함수(function), 산술식(arithmetic expression), 조건식(conditional expression)으로 이루어진
FAC언어의 문법구조와 의미구조를 그림 \ref{fig:FAClang}와 같이 정의하자.
함수를 다루는 규칙은 이전에 살펴본 타입없는 람다계산법에 대한
인자 먼저 값계산의 큰걸음 의미구조(그림 \ref{fig:bigStepCBV})와 같다.
산술식을 다루는 규칙도 이전에 살펴본 산술식에 대한 큰걸음 의미구조(\ref{sec:bigStepArith}절)와
사실상 같은 형태를 따르되 변수 이름이 산술식에 나타날 수도 있으므로
산술식($e_1 + e_2$)을 환경($\rho$)과 짝지어 계산한다.
조건식($\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0$)을 다루는 규칙은 
$e$를 계산한 결과값이 0인 경우와 그렇지 않은 경우로 나누어 두 개의 규칙으로 정의하고 있다.
즉, 0인 경우를 거짓으로 그 외의 정수값은 참으로 취급해 조건식의 계산을 진행하라는 뜻이다.

이 언어에서는 정상적으로 계산이 끝난 것으로 취급되는 값의 형태가 두 가지 있다.
함수를 계산한 결과값인 클로저($\langle\lambda x.e,\rho\rangle$)와
산술식을 계산한 결과값인 정수($n$) 이렇게 두 종류의 값이 있다는 말이다.

In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

In [2]:
type Nm = String  -- 변수 이름은 문자열로

data Expr = Var Nm             -- x
          | Lam Nm Expr        -- (\x.e)
          | App Expr Expr      -- (e1 e2)
          | Lit Int            -- n
          | Add Expr Expr      -- e1 + e2
          | If Expr Expr Expr  -- if e then e1 else e0
          deriving (Eq, Ord)

하스켈로 인터프리터를 작성하기 위한 문법구조를 위와 같이 선언한다.

In [3]:
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Expr where display = dTeX

In [4]:
idExpr = Lam "x" (Var "x")
ttExpr = Lam "x" (Lam "y" (Var "x")) 
ffExpr = Lam "x" (Lam "y" (Var "y")) 
-- 람다식, 산술식과, 조건식이 모두 활용된 예시 프로그램
If (Add (Lit 3) (Add (Var "x") (Var "y"))) ffExpr (App idExpr ttExpr)

In [5]:
data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]

의미구조 작성을 위한 두 종류의 값과 환경은 위와 같이 선언한다.

In [6]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,cl) = x++"{\\mapsto}"++show cl
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "

instance {-# OVERLAPS #-} IHaskellDisplay Env  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val)
                                               where display = dTeX
instance IHaskellDisplay Val                   where display = dTeX

지금까지는 비결정적이든 결정적이든 합류성이 있건 없건 관계없이 의미구조를 하스켈로
옮길 때 의미구조를 계산하는 함수의 결과를 항상 일관되게 리스트 타입으로 선언했다.
비결정적이거나 합류성이 없는 의미구조는 여러가지 계산 결과가 가능하므로
리스트 타입으로 선언하는 것이 적합하다. 그런데, 결정적 작은걸음 의미구조나
합류성이 있는 큰걸음 의미구조 함수는 계산 과정이 정상적으로 진행/종료되는 경우
하나의 유일한 결과만을 계산한다. 그래서, 이 장에서 작성하는 큰걸음 의미구조를
구현한 인터프리터 함수 `eval`은 하나의 값(`Val`)을 결과로 계산하도록 선언하였으며
계산이 정상적으로 진행되지 않아 비정상 종료하는 경우에는 하스켈 프로그램을 비정상
종료시키는 `error` 함수를 활용해 에러 메시지와 함께 비정상 종료되도록 처리하여
다음과 같이 작성하였다.

In [7]:
eval :: Expr -> Env -> Val   -- FAC언어의 인터프리터를 구현한 eval 함수
eval (Var x) env =
  case lookup x env of Nothing -> error (x ++ " not defined")
                       Just v  -> v
eval e@(Lam _ _) env = Cl e env
eval (App e1 e2) env =
  case v1 of Cl (Lam x e) env1 -> eval e ((x,v2):env1)
             _                 -> error (show v1++" not closure")
  where v1 = eval e1 env 
        v2 = eval e2 env 
eval (Lit n)     _   = Vi n
eval (Add e1 e2) env =
  case (v1, v2) of (Vi n1, Vi n2) -> Vi (n1 + n2)
                   (Vi _ , _    ) -> error (show v2++" not int")
                   _              -> error (show v1++" not int")
  where v1 = eval e1 env
        v2 = eval e2 env
eval (If e e1 e0) env =
  case eval e env of Vi 0 -> eval e0 env
                     Vi _ -> eval e1 env
                     _    -> error (show e++" not int")

In [8]:
e1 = Lam "x" . Lam "y" $ If (Var "x") (Var "x" `Add` Var "y")
                                      (Var "y" `Add` Var "y")
e2 = Lit 2
e3 = Lit 3

eval e1 []
eval e2 []
eval e3 []

In [9]:
App e1 e2
eval (App e1 e2) []

In [10]:
App (App e1 e2) e3
eval (App (App e1 e2) e3) []

In [11]:
App (App e1 (Lit 0)) e3
eval (App (App e1 (Lit 0)) e3) []

In [12]:
eval (Var "x") []

: 

In [13]:
eval (Lit 3 `App` Lit 1) []

: 

In [14]:
eval (Lit 3 `Add` (Lam "x" $ Var "x")) []

: 

# 고정점 조합자를 활용한 재귀
\label{sec:Zcombinator}
\ref{sec:fixpoint}절에서 소개한 고정점 조합자 $Y=\lambda f.(\lambda x.f\,(x\,x))\,(\lambda x.f\,(x\,x))$는
인자 먼저(call-by-value) 값계산에서 활용하기에 곤란하다. 왜냐하면 $f\,(x\,x)$의 계산을 함수 $f$의 적용부터 먼저 할 수 없고
반드시 인자 부분의 자기 자신에 대한 적용식 $x\,x$를 더 이상 계산이 진행되지 않을 때까지 값으로 계산하려 하기 때문이다.
자기 자신에 대한 적용식은 많은 경우 무한한 계산 과정으로 확산(diverge)한다. 따라서 $Y$조합자를 활용한 재귀적 계산을
인자 먼저 계산법에서 사용하면 끝나지 않고 무한한 계산 과정으로 확산하는 경우가 많다. $Y$조합자를 활용한 똑같은 재귀적
계산이 적용 먼저(call-by-name) 계산법이나 필요한 만큼(call-by-need) 계산법에서라면 정상적으로 종료되는 경우라 하더라도 말이다.

다행히 인자 먼저 계산법에서도 유용한 다음과 같은 고정점 조합자가 있다.\vspace*{-1em}
$$Z ~=~ \lambda f.(\lambda x.f\,(\lambda z.(x\,x)\,z))(\lambda x.f\,(\lambda z.(x\,x)\,z))$$
$~$\vspace*{-2.5em}\newline
참고로, $Z$조합자는 $Y$조합자와 $\eta$동치다. 왜냐하면, $\lambda z.(x\,x)\,z \equiv_\eta x\,x$이기 때문이다.
위의 $Z$조합자를 앞서 하스켈로 선언한 함수, 산술식, 조건식으로 이루어진 언어의 문법구조 데이터 타입(`Expr`)으로 옮기면 다음과 같다.

In [15]:
cZ = Lam "f" (App e e) 
  where e = Lam "x" $ f `App` Lam "z" (App x x `App` z)
        f = Var "f"
        x = Var "x"
        z = Var "z"
cZ -- \f.(\x.f(\z.xxz))(\x.f(\z.xxz))

$~$\vspace*{-1ex}\newline\indent
이제 주어진 자연수 $i$에 대해 0부터 $i$까지의 총합을 구하는 재귀적 계산을 $Z$조합자를 활용해 FAC언어로 작성해 보자.
만일 FAC언어에서 재귀함수 정의가 허용되었더라면 다음과 같이 작성할 수 있을 것이다.\vspace*{-1em}
$$f~i ~=~ \textbf{if}~i~\textbf{then}~i + f\,(i+({-}1))~\textbf{else}~i$$
$~$\vspace*{-2.5em}\newline
앞서 \ref{sec:fixHaskell}절에서 고정점을 재귀적이지 않은 함수에 적용하여 재귀적으로 정의했던 팩토리얼 함수와 마찬가지 계산을 하는 함수를 표현할 수 있다는 사례를 알아보았다.
마찬가지 방법으로 파라메터 $f$와 $i$에 대한 인자에 연달아 적용할 수 있는 아래와 같은 FAC언어로 작성된 람다식 `e4`를 선언하여 고정점 조합자와 함께 활용하면 된다.

In [16]:
e4 = Lam "f" . Lam "i" $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- \f.\i.if i then i + f(i + -1) else i

고정점 조합자 $Z$를 FAC언어로 옮긴 `cZ`를 위의 람다식 `e4`에 적용한 `App cZ e4`가
바로 우리가 정의하려던 주어진 자연수에 대한 총합을 재귀적으로 계산하는 함수를 FAC언어로 표현한 것이다.
실제로, 0에서 주어진 정수 `i`까지의 총합을 계산하는 FAC언어로 작성된 함수(`App cZ e4`)를
아래와 같이 100까지의 합을 구하도록 테스트해 보면 예상대로 0부터 100까지의 합인 5050이 계산됨 확인할 수 있다.

In [17]:
-- Z (\f.\i.if i then i + f(i + -1) else i) 100
eval (App cZ e4 `App` Lit 100) [] -- 100부터 0까지의 총합

\section*{연습문제}
1. 아직 고정점 조합자의 활용이 정확히 이해되지 않는다면 0부터 2까지의 총합을 구하는 식
   $Z~(\lambda f.\lambda i.\textbf{if}~i~\textbf{then}~i+f\,(i+({-}1))~\textbf{else}~i)~2$의 계산 과정을
   (치환 기반의 작은걸음 의미구조처럼) 한 단계씩 손으로 전개해 보라.

\begin{comment}
100은 너무 길어지니까 2정도로만 람다식을 손으로 전개해 보자.
```
cZ e4 2
= (\f.(\x.f(\z.xxz)) (\x.f(\z.xxz))) e4 2
= (\x.e4(\z.xxz)) (\x.e4(\z.xxz)) 2    여기서 w = (\x.e4(\z.xxz))라고 하자
= (\x.e4(\z.xxz)) w 2
= e4 (\z.w w z) 2
= (\f.\i.if i then i + f (i + -1) else i) (\z.w w z) 2
= (\i.if i then i + (\z.w w z) (i + -1) else i) 2
= if 2 then 2 + (\z.w w z) (2 + -1) else 2
= 2 + (\z.w w z) 1
= 2 + w w 1
= 2 + (\x.e4(\z.xxz)) w 1
= 2 + e4 (\z.w w z) 1
= 2 + (\f.\i.if i then i + f (i + -1) else i) (\z.w w z) 1
= 2 + (\i.if i then i + (\z.w w z) (i + -1) else i) 1
= 2 + if 1 then 1 + (\z.w w z) (1 + -1) else 1
= 2 + 1 + (\z.w w z) (1 + -1)
= 2 + 1 + w w 0
= 2 + 1 + (\x.e4(\z.xxz)) w 0
= 2 + 1 + e4 (\z.w w z) 0
= 2 + 1 + (\f.\i.if i then i + f (i + -1) else i) (\z.w w z) 0
= ...
= 2 + 1 + if 0 then 0 + f (0 + -1) else 0
= 2 + 1 + 0
```
\end{comment}